In [ ]:
import vector
from tqdm.auto import tqdm
import awkward as ak

import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

import numpy as np

import glob

import sys
sys.path += ["../../mlpf/plotting/"]
from plot_utils import experiment_label, sample_label, EVALUATION_DATASET_NAMES

In [ ]:
!ls ../../experiments//pyg-cms_20250722_101813_274478/preds_checkpoint-10-3.812332/

In [ ]:
which_process = 'cms_pf_ztt'
#modify path as appropriate
files = list(glob.glob('../../experiments/pyg-cms_20250722_101813_274478/preds_checkpoint-10-3.812332/{}/*.parquet'.format(which_process)))[:1000]
arr = ak.concatenate([ak.from_parquet(file) for file in tqdm(files)])

In [ ]:
particle_types = {
    'ch. had.': 1,
    'n. had.': 2,
    '$e^\pm$': 6,
    '$\mu^\pm$': 7,
    '$\gamma$': 5,
    'HF had.': 4,
    'HF e.m.': 3,
}

particle_types_literal = {
    'ch. had.': 'chhad',
    'n. had.': 'nhad',
    '$e^\pm$': 'e',
    '$\mu^\pm$': 'mu',
    '$\gamma$': 'gamma',
    'HF had.': 'hfhad',
    'HF e.m.': 'hfem',
}

pu_target_pred = {}

for particle_type in particle_types:
    sel = (arr.particles.target.cls_id == particle_types[particle_type])
    sel1 = (arr.particles.target.pt > 5) & (arr.particles.target.pt < 10)
    pu_target_pred[particle_type] = [arr.particles.target[sel & sel1].ispu, arr.particles.pred[sel & sel1].ispu]

In [ ]:
import mplhep as hep

fig = plt.figure(figsize=(10,10))
ax = plt.axes()
plt.style.use(hep.style.CMS)

for particle_type in particle_types:
    fpr, tpr, thresholds = roc_curve(ak.flatten(pu_target_pred[particle_type][0]) == 1, ak.flatten(pu_target_pred[particle_type][1]))
    j_statistic = tpr-fpr
    thresh_idx = np.argmax(j_statistic)
    print(particle_type, "thresh={:.2f}".format(thresholds[thresh_idx]))

    auc = roc_auc_score(ak.flatten(pu_target_pred[particle_type][0]) == 1, ak.flatten(pu_target_pred[particle_type][1]))
    ls = "-"
    if particle_type in ["HF had.", "HF e.m."]:
        ls = "--"
    c = plt.plot(fpr, tpr, label = particle_type+', AUC=%.2f, $p^{max}_{PU}$=%.2f'%(auc, thresholds[thresh_idx]), linewidth=2.5, ls=ls)
    plt.plot([fpr[thresh_idx]], [tpr[thresh_idx]], marker="o", color=c[0].get_color(), markersize=10)

plt.plot([0, 1], [0, 1],'k--', label='AUC=0.5')
plt.legend(loc = 'lower right', frameon=False, title=EVALUATION_DATASET_NAMES[which_process] + "\n$p_{T,reco}$ > 1 GeV", fontsize=16, title_fontsize=24)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True positive rate', loc='top')
plt.xlabel('False positive rate', loc='right')

#cms_label(ax)
hep.cms.label("Preliminary", data=False, com=14, year='Run 3')

#modify path as appropriate
plt.savefig(f'pu_performance/{which_process}_pu_roc.pdf')

In [ ]:
import mplhep as hep

fig = plt.figure()
ax = plt.axes()
plt.style.use(hep.style.CMS)

#'$\gamma$', 
for particle_type, ls in zip(['$\gamma$', 'n. had.'], ["-", "-"]):
    fpr, tpr, thresholds = roc_curve(ak.flatten(pu_target_pred[particle_type][0]) == 1, ak.flatten(pu_target_pred[particle_type][1]))
    thresh_idx = len(thresholds) - np.searchsorted(thresholds[::-1], 0.1)
    auc = roc_auc_score(ak.flatten(pu_target_pred[particle_type][0]) == 1, ak.flatten(pu_target_pred[particle_type][1]))
    c = plt.plot(fpr, tpr, label = particle_type+', AUC = %0.3f' % auc, linewidth=2.5, ls=ls)
    plt.plot([fpr[thresh_idx]], [tpr[thresh_idx]], marker="o", color=c[0].get_color(), markersize=10)
    
    target_pt = arr.particles.target.pt[
        (arr.particles.target.cls_id == particle_types[particle_type])
        & (arr.particles.target.pt>1)& (arr.particles.target.pt<5)
    ]
    print(
        len(ak.flatten(pu_target_pred[particle_type][0])),
        len(ak.flatten(target_pt))
    )
    fpr, tpr, thresholds = roc_curve(ak.flatten(pu_target_pred[particle_type][0]) == 1, -ak.flatten(target_pt))
    c = plt.plot(fpr, tpr, linewidth=2.5, color=c[0].get_color(), ls="--")


plt.plot([0, 1], [0, 1],'k--', label='AUC=0.5')
plt.legend(loc = 'lower right', frameon=False, title=EVALUATION_DATASET_NAMES[which_process] + "\n$p_{T,reco}$ > 1 GeV", fontsize=16, title_fontsize=24)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True positive rate', loc='top')
plt.xlabel('False positive rate', loc='right')

#cms_label(ax)
hep.cms.label("Preliminary", data=False, com=14, year='Run 3')

#modify path as appropriate
plt.savefig(f'pu_performance/{which_process}_pu_roc_reduced.pdf')

In [ ]:
plt.hist2d(
    ak.to_numpy(ak.flatten(pu_target_pred[particle_type][1])),
    ak.to_numpy(ak.flatten(target_pt)),
    bins=(np.linspace(0,1,100), np.logspace(0,3,100)),
    norm=matplotlib.colors.LogNorm()
)
plt.yscale("log")

In [ ]:
for particle_type in pu_target_pred:

    plt.style.use(hep.style.CMS)

    fig = plt.figure()
    ax = plt.gca()

    bins = np.linspace(0,1,100)
    plt.hist(ak.flatten(pu_target_pred[particle_type][1][pu_target_pred[particle_type][0]==0]), histtype='step', bins=bins, label='non-PU', linewidth=2.5, density=1)
    plt.hist(ak.flatten(pu_target_pred[particle_type][1][pu_target_pred[particle_type][0]==1]), histtype='step', bins=bins, label='PU', linewidth=2.5, density=1)

    ax.set_yscale('log')
    ax.set_xlabel("MLPF pileup pred. ", loc='right')
    ax.set_ylabel('Particles', loc='top')

    hep.cms.label("Preliminary", data=False, com=14, year='Run 3')
    
    plt.legend(title=EVALUATION_DATASET_NAMES[which_process]+'\n'+particle_type, frameon=False, loc=1)
    plt.ylim(top=ax.get_ylim()[1]*100)
    
    #modify path as appropriate
    plt.savefig(f'pu_performance/{which_process}_{particle_types_literal[particle_type]}_pu.pdf')

In [ ]:
bins = np.linspace(0,1,100)

pu_target_pred = {}

for particle_type in ['n. had.']:
    sel = (arr.particles.target.cls_id == particle_types[particle_type])
    sel1 = (arr.particles.target.pt > 1) & (arr.particles.target.pt < 5)
    pu_target_pred[particle_type] = [arr.particles.target[sel & sel1].ispu, arr.particles.pred[sel & sel1].ispu]
    
particle_type = 'n. had.'
plt.hist(ak.flatten(pu_target_pred[particle_type][1][pu_target_pred[particle_type][0]==0]), histtype='step', bins=bins, label='non-PU', linewidth=2.5, density=1)
plt.hist(ak.flatten(pu_target_pred[particle_type][1][pu_target_pred[particle_type][0]==1]), histtype='step', bins=bins, label='PU', linewidth=2.5, density=1);
plt.legend(title="n.had., 1<pt<5")

In [ ]:
bins = np.linspace(0,1,100)

pu_target_pred = {}

for particle_type in ['n. had.']:
    sel = (arr.particles.target.cls_id == particle_types[particle_type])
    sel1 = (arr.particles.target.pt > 5) & (arr.particles.target.pt < 10)
    pu_target_pred[particle_type] = [arr.particles.target[sel & sel1].ispu, arr.particles.pred[sel & sel1].ispu]
    
particle_type = 'n. had.'
plt.hist(ak.flatten(pu_target_pred[particle_type][1][pu_target_pred[particle_type][0]==0]), histtype='step', bins=bins, label='non-PU', linewidth=2.5, density=1)
plt.hist(ak.flatten(pu_target_pred[particle_type][1][pu_target_pred[particle_type][0]==1]), histtype='step', bins=bins, label='PU', linewidth=2.5, density=1);
plt.legend(title="n.had., 5<pt<10")

In [ ]:
bins = np.linspace(0,1,100)

pu_target_pred = {}

for particle_type in ['n. had.']:
    sel = (arr.particles.target.cls_id == particle_types[particle_type])
    sel1 = (arr.particles.target.pt > 10) & (arr.particles.target.pt < 20)
    pu_target_pred[particle_type] = [arr.particles.target[sel & sel1].ispu, arr.particles.pred[sel & sel1].ispu]
    
particle_type = 'n. had.'
plt.hist(ak.flatten(pu_target_pred[particle_type][1][pu_target_pred[particle_type][0]==0]), histtype='step', bins=bins, label='non-PU', linewidth=2.5, density=1)
plt.hist(ak.flatten(pu_target_pred[particle_type][1][pu_target_pred[particle_type][0]==1]), histtype='step', bins=bins, label='PU', linewidth=2.5, density=1);
plt.legend(title="n.had., 50<pt<100")

In [ ]:
ptbins = np.logspace(0,2,21)
threshes = {
    2: [],
    3: [],
    4: [],
    5: [],
}
for ibin in range(len(ptbins)-1):

    for cls_id in [2,3,4,5]:
        sel = (arr.particles.target.cls_id == cls_id)
        sel1 = (arr.particles.target.pt >= ptbins[ibin]) & (arr.particles.target.pt < ptbins[ibin+1])
        tgt = arr.particles.target[sel & sel1].ispu
        pred = arr.particles.pred[sel & sel1].ispu

        if len(tgt)>0 and np.sum(tgt)>0:
            fpr, tpr, thresh = roc_curve(ak.flatten(tgt) == 1, ak.flatten(pred))
            idx = np.searchsorted(fpr, 0.8)
            threshes[cls_id].append(thresh[idx])
        else:
            threshes[cls_id].append(0)

In [ ]:
plt.plot(ptbins[:-1], threshes[2], marker="o", label="n.had")
plt.plot(ptbins[:-1], threshes[3], marker="o", label="HF e.m")
plt.plot(ptbins[:-1], threshes[4], marker="o", label="HF had.")
plt.plot(ptbins[:-1], threshes[5], marker="o", label="gamma")
plt.xscale("log")
plt.xlabel("target pt")
plt.ylabel("threshold @ FPR 80%")
plt.legend()